In [31]:
import argparse
import time

from PIL import Image
from PIL import ImageDraw

import detect
import tflite_runtime.interpreter as tflite
import platform
from pathlib import Path
import urllib.request

In [32]:
EDGETPU_SHARED_LIB = {
  'Linux': 'libedgetpu.so.1',
  'Darwin': 'libedgetpu.1.dylib',
  'Windows': 'edgetpu.dll'
}[platform.system()]

In [33]:
def load_labels(path, encoding='utf-8'):
  """Loads labels from file (with or without index numbers).

  Args:
    path: path to label file.
    encoding: label file encoding.
  Returns:
    Dictionary mapping indices to labels.
  """
  with open(path, 'r', encoding=encoding) as f:
    lines = f.readlines()
    if not lines:
      return {}

    if lines[0].split(' ', maxsplit=1)[0].isdigit():
      pairs = [line.split(' ', maxsplit=1) for line in lines]
      return {int(index): label.strip() for index, label in pairs}
    else:
      return {index: line.strip() for index, line in enumerate(lines)}

In [34]:
def make_interpreter(model_file):
  model_file, *device = model_file.split('@')
  return tflite.Interpreter(
      model_path=model_file,
      experimental_delegates=[
          tflite.load_delegate(EDGETPU_SHARED_LIB,
                               {'device': device[0]} if device else {})
      ])

In [35]:
def draw_objects(draw, objs, labels):
  """Draws the bounding box and label for each object."""
  for obj in objs:
    bbox = obj.bbox
    draw.rectangle([(bbox.xmin, bbox.ymin), (bbox.xmax, bbox.ymax)],
                   outline='red')
    draw.text((bbox.xmin + 10, bbox.ymin + 10),
              '%s\n%.2f' % (labels.get(obj.id, obj.id), obj.score),
              fill='red')

In [36]:
Path("models").mkdir(parents=True, exist_ok=True)
Path("images").mkdir(parents=True, exist_ok=True)

In [37]:
url = 'https://github.com/google-coral/edgetpu/raw/master/test_data/mobilenet_ssd_v2_coco_quant_postprocess_edgetpu.tflite'
urllib.request.urlretrieve(url, 'models/mobilenet_ssd_v2_coco_quant_postprocess_edgetpu.tflite')
url = 'https://github.com/google-coral/edgetpu/raw/master/test_data/mobilenet_ssd_v2_coco_quant_postprocess.tflite'
urllib.request.urlretrieve(url, 'models/mobilenet_ssd_v2_coco_quant_postprocess.tflite')
url = 'https://github.com/google-coral/edgetpu/raw/master/test_data/coco_labels.txt'
urllib.request.urlretrieve(url, 'models/coco_labels.txt')
url = 'https://github.com/google-coral/edgetpu/raw/master/test_data/grace_hopper.bmp'
urllib.request.urlretrieve(url, 'images/grace_hopper.bmp')

('images/grace_hopper.bmp', <http.client.HTTPMessage at 0x233aee53708>)

In [38]:
labels = load_labels('models/coco_labels.txt')
interpreter = make_interpreter('models/mobilenet_ssd_v2_coco_quant_postprocess.tflite')
interpreter.allocate_tensors()

In [39]:
image = Image.open('images/grace_hopper.bmp')
scale = detect.set_input(interpreter, image.size,
                       lambda size: image.resize(size, Image.ANTIALIAS))

In [40]:
print('----INFERENCE TIME----')
print('Note: The first inference is slow because it includes',
    'loading the model into Edge TPU memory.')

----INFERENCE TIME----
Note: The first inference is slow because it includes loading the model into Edge TPU memory.


In [41]:
for _ in range(5):
    start = time.perf_counter()
    interpreter.invoke()
    inference_time = time.perf_counter() - start
    objs = detect.get_output(interpreter, 0.4, scale)
    print('%.2f ms' % (inference_time * 1000))

619.49 ms
667.31 ms
624.05 ms
652.16 ms
630.06 ms


In [42]:
print('-------RESULTS--------')
if not objs:
    print('No objects detected')

-------RESULTS--------


In [43]:
for obj in objs:
    print(labels.get(obj.id, obj.id))
    print('  id:    ', obj.id)
    print('  score: ', obj.score)
    print('  bbox:  ', obj.bbox)

person
  id:     0
  score:  0.83984375
  bbox:   BBox(xmin=2, ymin=5, xmax=507, ymax=590)
tie
  id:     31
  score:  0.83984375
  bbox:   BBox(xmin=225, ymin=417, xmax=291, ymax=539)


In [44]:
image = image.convert('RGB')
draw_objects(ImageDraw.Draw(image), objs, labels)
image.save('images/output.jpg')
image.show()